In [1]:
import importlib
import pandas as pd
import nltk
import gensim 
import logging

import utils
importlib.reload(utils)

<module 'utils' from '/home/andryw/Projects/tutorials/embeddings/utils.py'>

In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)


In [3]:
#utils.normalize_text(sentence, remove stopwords=false) without sample=1e-3 window=5 
model.wv.most_similar(positive=["woman", "king"], negative=["man"])

/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.518113374710083),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]

### Load data to predict

In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import xgboost as xgb

In [7]:
data = pd.read_csv("data/labeledTrainData.tsv", delimiter="\t", quoting=3)
train_data, test_data = train_test_split(data, test_size=0.25)

In [8]:
train_data['review_cleaned'] = train_data.review.apply(lambda x: utils.normalize_text(x))
test_data['review_cleaned'] = test_data.review.apply(lambda x: utils.normalize_text(x))

/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
def get_avg_vectors(text, model):
    total_words = 0
    final_vector = np.zeros(model.vector_size)
    for word in text.split(" "):
        if word in model.vocab:
            final_vector += model.get_vector(word)
            total_words+=1
    return final_vector / total_words



In [11]:
train_vectors = [get_avg_vectors(x, model.wv) for x in train_data['review_cleaned']]
train_vectors=np.array(train_vectors)

test_vectors = [get_avg_vectors(x, model.wv) for x in test_data['review_cleaned']]
test_vectors=np.array(test_vectors)

/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [12]:
x_train = train_vectors
y_train = train_data.sentiment

x_test = test_vectors
y_test  = test_data.sentiment

In [13]:
logistic_regression = LogisticRegression(verbose=True, n_jobs=3)
logistic_regression = logistic_regression.fit(x_train, y_train)
metrics.roc_auc_score(logistic_regression.predict(x_test), y_test)

/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear]

0.8587160734837332

In [14]:
# params = {"n_jobs": 3, "silent": True, "n_estimators": 100, 'objective':'binary:logistic'}
# xgboost_classifier = xgb.XGBClassifier(**params)

# xgboost_classifier.fit(x_train, y_train, verbose=True, eval_metric='auc',
#                       eval_set=[(x_train, y_train), (x_test, y_test)],)

# metrics.roc_auc_score(xgboost_classifier.predict(x_test), y_test)

## Predict

In [16]:
unlabel_data = pd.read_csv("data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabel_data['review_cleaned'] = unlabel_data.review.apply(lambda x: utils.normalize_text(x))
unlabel_vectors = [get_avg_vectors(x, model.wv) for x in unlabel_data['review_cleaned']]
unlabel_vectors=np.array(unlabel_vectors)

/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
utils.save_predict(unlabel_data, 
                   logistic_regression.predict(unlabel_vectors), 
                   "logistic_word2vec-pretrained.csv")